In [9]:
EXPORT_TO_JSON = False
EXPORT_TO_SQLITE = True
DATABASE_NAME = "../db.sqlite"

In [10]:
# Import required libraries
import overpy
import json
import sqlite3

from dataclasses import dataclass
from decimal import Decimal
from contextlib import closing

api = overpy.Overpass()

#### Declare `TramStop` class

In [11]:
@dataclass(frozen=True)
class TramStop:
    """
        TramStop class represents a tram stop. Tram stops are considered equal
        when their names are the same. In case of multiple stops with the same name,
        their locations should be close enough to not make a difference in the model
        and they would be difficult to distinguish between each other using the
        data available from the time-table.
    """

    node_id: int
    name: str
    latitude: Decimal
    longitude: Decimal
    
    def __hash__(self) -> int:
        return hash(self.name)
    
    def __eq__(self, other) -> bool:
        if not isinstance(other, TramStop):
            return False
        
        return self.name == other.name
    
    def to_json(self) -> dict[str, int | str | tuple[float, float]]:
        return {
            "id": self.node_id,
            "name": self.name,
            "latitude": float(self.latitude),
            "longitude": float(self.longitude)
        }
    
    def to_sql_parameters(self) -> tuple[int, str, float, float]:
        return (
            self.node_id,
            self.name,
            float(self.latitude),
            float(self.longitude)
        )

#### Execute query using Overpass Turbo

In [12]:
tram_stops_query_result = api.query("""
    [out:json];
    area["name"="Kraków"]->.search_area;
    node["tram"="yes"]["railway"="tram_stop"](area.search_area);
    out;
""")

#### Create a set of all tram stops

In [13]:
tram_stops = {TramStop(item.id, item.tags.get("name"), item.lat, item.lon) for item in tram_stops_query_result.nodes}

#### Export to JSON

In [14]:
if EXPORT_TO_JSON:
    with open("tram-stops.json", "w+") as file:
        json.dump([item.to_json() for item in tram_stops], file, indent=4, ensure_ascii=False)

#### Export to SQLite3

In [15]:
if EXPORT_TO_SQLITE:
    with closing(sqlite3.connect(DATABASE_NAME)) as connection, closing(connection.cursor()) as cursor:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS tram_stops (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL UNIQUE ON CONFLICT REPLACE,
                latitude DECIMAL(10, 7) NOT NULL,
                longitude DECIMAL(10, 7) NOT NULL
            );
        """)

        cursor.executemany(
            "INSERT INTO tram_stops VALUES (?, ?, ?, ?) ON CONFLICT (id) DO NOTHING",
            (item.to_sql_parameters() for item in tram_stops)
        )

        connection.commit()